In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler


In [2]:
#{"DX": [], "DY": [], "DZ": [], "H": [], "D1": [], "D2": [], "log_f": [], "RP": []})
NLOS_DATA = pd.read_csv("../csv/7-14-1.csv")
NLOS_DATA.pop("Unnamed: 0")

ROW_NUM = len(NLOS_DATA.index)
TRAINING_NUM = int(ROW_NUM / 5 * 3)
VALIDATION_NUM = int(ROW_NUM / 5 )

TRAINING_DATA = NLOS_DATA.iloc[ :TRAINING_NUM,:]
VALIDATION_DATA = NLOS_DATA.iloc[TRAINING_NUM:TRAINING_NUM+VALIDATION_NUM,:]
TEST_DATA = NLOS_DATA.iloc[TRAINING_NUM+VALIDATION_NUM:,:]
TRAINING_VAL = pd.DataFrame(TRAINING_DATA.pop('RP'))
VALIDATION_VAL = pd.DataFrame(VALIDATION_DATA.pop('RP'))
TEST_VAL = pd.DataFrame(TEST_DATA.pop('RP'))


In [3]:
def scaling(DATA):
    min_max_scaler = MinMaxScaler()
    fitted = min_max_scaler.fit(DATA)
    DATA = min_max_scaler.transform(DATA)
    return pd.DataFrame(DATA)
    
TRAINING_DATA = scaling(TRAINING_DATA)
VALIDATION_DATA = scaling(VALIDATION_DATA)
TEST_DATA = scaling(TEST_DATA)

In [4]:
model = keras.models.Sequential([
    keras.layers.Dense(14,activation="sigmoid", input_dim=7),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="mean_squared_error", optimizer='sgd')


2022-05-10 16:37:55.187019: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model.fit(TRAINING_DATA, TRAINING_VAL, epochs=100, batch_size=100, validation_data=(VALIDATION_DATA, VALIDATION_VAL))


Epoch 1/100
336/336 [==============================] - 1s 1ms/step - loss: 0.0548 - val_loss: 0.0562
Epoch 2/100
336/336 [==============================] - 0s 921us/step - loss: 0.0522 - val_loss: 0.0537
Epoch 3/100
336/336 [==============================] - 0s 749us/step - loss: 0.0496 - val_loss: 0.0509
Epoch 4/100
336/336 [==============================] - 0s 748us/step - loss: 0.0468 - val_loss: 0.0478
Epoch 5/100
336/336 [==============================] - 0s 738us/step - loss: 0.0438 - val_loss: 0.0445
Epoch 6/100
336/336 [==============================] - 0s 767us/step - loss: 0.0404 - val_loss: 0.0409
Epoch 7/100
336/336 [==============================] - 0s 767us/step - loss: 0.0368 - val_loss: 0.0370
Epoch 8/100
336/336 [==============================] - 0s 796us/step - loss: 0.0328 - val_loss: 0.0327
Epoch 9/100
336/336 [==============================] - 0s 722us/step - loss: 0.0287 - val_loss: 0.0283
Epoch 10/100
336/336 [==============================] - 0s 733us/step - los

In [6]:
scores = model.evaluate(TEST_DATA, TEST_VAL, batch_size=20)
print(scores)


559/559 [==============================] - 0s 514us/step - loss: 0.0016
0.0015621158527210355


In [7]:
predictions = pd.DataFrame(model.predict(TEST_DATA))
predictions.columns=["PREDICITONS"]
TEST_VAL.columns=["TEST_VAL"]
result=pd.concat([predictions, TEST_VAL], axis=1)
result.iloc[:30,:].to_csv("7-14-1.csv")
